In [6]:
import pickle
import pandas as pd

In [17]:
df=pd.read_pickle('deception_df.pkl')


In [24]:
print('most unpolite messages')
display(df.sort_values('politeness').head(5).messages)
print('most polite messages')
df.sort_values('politeness', ascending=False).head(5).messages

most unpolite messages


2947                             I will not fuck you over
9493    What a clod you are. How in blue fuck does kee...
2945                               Don’t fuck me over tho
5760    You're a dick dude.  I will make sure that tha...
9629    I was getting you into Kiel. You made a bad de...
Name: messages, dtype: object

most polite messages


15767    Thanks, much appreciated - Hoping to work toge...
6958                                        Ok, thank you!
11526                         That would be great! Thanks!
491       Okay, thank you for letting me know. That helps!
11599             Sounds great man! Thanks for everything!
Name: messages, dtype: object

In [ ]:
print('most unpolite messages')
display(df.sort_values('politeness').head(5).messages)
print('most polite messages')
df.sort_values('politeness', ascending=False).head(5).messages